In [1]:
import numpy as np  
import numpy.polynomial.legendre as npl
import sympy as sp
import matplotlib.pyplot as plt
from scipy.special import legendre

In [2]:
def Transformation(nodes, weight,a,b):
    nodes=(b-a)/2*nodes+(a+b)/2
    weight=(b-a)/2 *weight
    return (nodes,weight)

In [3]:
def chop(lista):
    for i in range(len(lista)):
        if(abs(lista[i])<(10**(-10))):
            lista[i]=0
    return lista

In [4]:
class Cuadrature:
    def __init__(self,nodes=None,weight=None):
        if(list(nodes)!=None):
                self.nodes=nodes
        if(list(weight)!=None):
                self.weight=weight
        
    def getNodes(self):
        return self.nodes
    def getWeight(self):
        return self.weight
    def getNumPoints(self):
        return len(self.nodes)
    
    def Integrate(self,f):
        return self.Integrate_List(f(np.array(self.nodes)))
    
    def Integrate_List(self,lista):
        if (len(lista)==len(self.weight)):
            return np.array(lista).dot(np.array(self.weight))
        else:
            raise Exception("No has metido correctamente la lista")

In [5]:
class Lobato(Cuadrature):
    def __init__(self, num_points,a,b):
        aux=[-1,*sorted(list(legendre(num_points).deriv().r)),1]
        Cuadrature.__init__(self, nodes=np.array(aux), weight=np.array(2/(num_points*(num_points+1)*(legendre(num_points)(aux))**2)))
        self.nodes,self.weight=Transformation(self.nodes, self.weight, a, b)
    def __str__(self):
        return "Nodos: %s \nPesos: %s " %(self.nodes,self.weight)

In [6]:
class Legendre(Cuadrature):
    def __init__(self,num_points,a,b):
        Cuadrature.__init__(self,nodes=npl.leggauss(num_points)[0], weight=npl.leggauss(num_points)[1]) 
        self.nodes,self.weight=Transformation(self.nodes, self.weight, a, b)
    def __str__(self):
        return "Nodos: %s \nPesos: %s " %(self.nodes,self.weight)

In [7]:
class Trapecio_Compuesto(Cuadrature): #están considerados los extremos(-1,1)
    def __init__(self,num_points,a,b):
        Cuadrature.__init__(self, nodes=np.array([*np.arange(-1,1,float(2/num_points)).tolist(),1]), weight=np.array([2/(2*num_points),*[2/num_points]*(num_points-1),2/(2*num_points)]))
        self.nodes,self.weight=Transformation(self.nodes, self.weight, a, b)
    def __str__(self):
        return "Nodos: %s \nPesos: %s " %(self.nodes,self.weight)

In [19]:
N=4
x = sp.symbols('x')
Ln = lambda x:chop(legendre(4))(x)
Lnp= lambda x:chop(legendre(4)).deriv()(x)
xvals = chop(Lobato(N,-1,1).getNodes())
print(xvals)

phi0 = lambda x: sp.simplify(((-1)/(N*(N+1)))*((1-x**2)*(Lnp(x))/((x-xvals[0])*(Ln(xvals)[0]))))
difphi0 = lambda x: sp.simplify(sp.diff(phi0(x)))
difphi0 = lambda x: 3.5*x**3-2.625*x**2-0.75*x+0.375
                                                                                    
phi1 = lambda x:((-1)/(N*(N+1)))*((1-x**2)*(Lnp(x))/((x-xvals[1])*(Ln(xvals[1]))))
difphi1 = lambda x: sp.simplify(sp.diff(phi1(x)))
difphi1 = lambda x: (-3.5*x**5-2.1875*x**4+2.5*x**3+1.875*x**2-0.1875)/(0.2890625*x**2+0.2890625*x+0.072265625)

phi2 = lambda x: sp.simplify(((-1)/(N*(N+1)))*((1-x**2)*(Lnp(x))/((x-xvals[2])*(Ln(xvals)[2]))))
difphi2 = lambda x: sp.simplify(sp.diff(phi2(x)))
difphi2 = lambda x: x*(9.33333333333333*x**2-6.66666666666667)

phi3 = lambda x: sp.simplify(((-1)/(N*(N+1)))*((1-x**2)*(Lnp(x))/((x-xvals[3])*(Ln(xvals)[3]))))
difphi3 = lambda x: sp.simplify(sp.diff(phi3(x)))
difphi3 = lambda x: (-3.5*x**5+2.1875*x**4+2.5*x**3-1.875*x**2+0.1875)/(0.2890625*x**2-0.2890625*x+0.072265625)

phi4 = lambda x: sp.simplify(((-1)/(N*(N+1)))*((1-x**2)*(Lnp(x))/((x-xvals[4])*(Ln(xvals)[4]))))
difphi4 = lambda x: sp.simplify(sp.diff(phi4(x)))
difphi4 = lambda x: 3.5*x**3+2.625*x**2-0.75*x-0.375

sp.simplify(phi1(x))

SyntaxError: invalid syntax (<ipython-input-19-191275b3057e>, line 12)

In [13]:
from fractions import Fraction

num,den=sp.fraction(phi1(x))
aux=sp.Poly(num,x).all_coeffs()
[Fraction.from_float(float(i)) for i in aux]

[Fraction(7, 8),
 Fraction(0, 1),
 Fraction(-5, 4),
 Fraction(0, 1),
 Fraction(3377699720527873, 9007199254740992),
 Fraction(0, 1)]

In [20]:
def Matriz_Local(m,p,Nq,num_interval): #p siempre será 4 en nuestra actividad
    
    intervals=np.linspace(-1,1,m+1)
    A=np.zeros((p+1, p+1))
    a=intervals[num_interval-1]
    b=intervals[num_interval]
    xvals2=np.linspace(a,b,Nq)
    
    A[0][0]=Lobato(Nq-1,a,b).Integrate_List(difphi0(xvals2)*difphi0(xvals2))
    A[0][1]=Lobato(Nq-1,a,b).Integrate_List(difphi0(xvals2)*difphi1(xvals2))
    A[0][2]=Lobato(Nq-1,a,b).Integrate_List(difphi0(xvals2)*difphi2(xvals2))
    A[0][3]=Lobato(Nq-1,a,b).Integrate_List(difphi0(xvals2)*difphi3(xvals2))
    A[0][4]=Lobato(Nq-1,a,b).Integrate_List(difphi0(xvals2)*difphi4(xvals2))

    A[1][0]=Lobato(Nq-1,a,b).Integrate_List(difphi1(xvals2)*difphi0(xvals2))
    A[1][1]=Lobato(Nq-1,a,b).Integrate_List(difphi1(xvals2)*difphi1(xvals2))
    A[1][2]=Lobato(Nq-1,a,b).Integrate_List(difphi1(xvals2)*difphi2(xvals2))
    A[1][3]=Lobato(Nq-1,a,b).Integrate_List(difphi1(xvals2)*difphi3(xvals2))
    A[1][4]=Lobato(Nq-1,a,b).Integrate_List(difphi1(xvals2)*difphi4(xvals2))

    A[2][0]=Lobato(Nq-1,a,b).Integrate_List(difphi2(xvals2)*difphi0(xvals2))
    A[2][1]=Lobato(Nq-1,a,b).Integrate_List(difphi2(xvals2)*difphi1(xvals2))
    A[2][2]=Lobato(Nq-1,a,b).Integrate_List(difphi2(xvals2)*difphi2(xvals2))
    A[2][3]=Lobato(Nq-1,a,b).Integrate_List(difphi2(xvals2)*difphi3(xvals2))
    A[2][4]=Lobato(Nq-1,a,b).Integrate_List(difphi2(xvals2)*difphi4(xvals2))

    A[3][0]=Lobato(Nq-1,a,b).Integrate_List(difphi3(xvals2)*difphi0(xvals2))
    A[3][1]=Lobato(Nq-1,a,b).Integrate_List(difphi3(xvals2)*difphi1(xvals2))
    A[3][2]=Lobato(Nq-1,a,b).Integrate_List(difphi3(xvals2)*difphi2(xvals2))
    A[3][3]=Lobato(Nq-1,a,b).Integrate_List(difphi3(xvals2)*difphi3(xvals2))
    A[3][4]=Lobato(Nq-1,a,b).Integrate_List(difphi3(xvals2)*difphi4(xvals2))

    A[4][0]=Lobato(Nq-1,a,b).Integrate_List(difphi4(xvals2)*difphi0(xvals2))
    A[4][1]=Lobato(Nq-1,a,b).Integrate_List(difphi4(xvals2)*difphi1(xvals2))
    A[4][2]=Lobato(Nq-1,a,b).Integrate_List(difphi4(xvals2)*difphi2(xvals2))
    A[4][3]=Lobato(Nq-1,a,b).Integrate_List(difphi4(xvals2)*difphi3(xvals2))
    A[4][4]=Lobato(Nq-1,a,b).Integrate_List(difphi4(xvals2)*difphi4(xvals2))
    
    return A

In [63]:
#m,p,Nq,num_interval

A1=Matriz_Local(6,4,5,1)#m=7 intervalos dentro del (-1,1)
A2=Matriz_Local(6,4,5,2) #las phi siguen siendo del 0 al 4 (5) grados de libertad
A3=Matriz_Local(6,4,5,3) #Consideramos 3 nodos de cuadratura
A4=Matriz_Local(6,4,5,4)
A5=Matriz_Local(6,4,5,5)
A6=Matriz_Local(6,4,5,6) #¿Existe algun nombre para cambiar de matriz y que se haga un for?

C:\Users\isabe\Desktop\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  
C:\Users\isabe\Desktop\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in true_divide


In [64]:
A=[A1,A2,A3,A4,A5,A6]
A[0][1][4]

-0.19423669243231573

In [59]:
A1

array([[ 3.11373337, -1.37353571,  0.31412673, -0.23511992,  0.04760573],
       [-1.37353571,  3.84908852, -1.14565545,  0.92518683, -0.19423669],
       [ 0.31412673, -1.14565545,  0.37619861, -0.31002267,  0.06567038],
       [-0.23511992,  0.92518683, -0.31002267,  0.25660178, -0.05446004],
       [ 0.04760573, -0.19423669,  0.06567038, -0.05446004,  0.01156838]])

In [ ]:
intervalo 1: matriz 5x5 0 0 -> 0 0
  (k=0)                 0 1 -> 0 1
                        .
                        .
                        4 4 -> 4 4
                    
intervalo 2: matriz 5x5 0 0 -> 4 4
 (k=1)                  0 1 -> 4 5
                        1 0 -> 5 4
                        .
                        .
                        4 4 -> 8 8
intervalo 3: matriz 5x5 0 0 -> 8 8
                        0 1 -> 8 9
                        1 0 -> 9 8
                        .
                        .
                        4 4 -> 12 12...               

In [79]:
def Matriz_Global(A,m,p):#m es nº de intervalos
        B=np.zeros(((p+1)*m, (p+1)*m))
        for k in range(0,m):
            for i in range(0,p+1):
                for j in range(0,p+1):
                    B[k*p+i][k*p+j] = B[k*p+i][k*p+j] + A[k][i][j]
                    print("int",k+1,"k*p+i:",k*p+i," k*p+j:",k*p+j," k,i,j",k,i,j)
        return B

In [81]:
B=Matriz_Global(A,6,4)

int 1 k*p+i: 0  k*p+j: 0  k,i,j 0 0 0
int 1 k*p+i: 0  k*p+j: 1  k,i,j 0 0 1
int 1 k*p+i: 0  k*p+j: 2  k,i,j 0 0 2
int 1 k*p+i: 0  k*p+j: 3  k,i,j 0 0 3
int 1 k*p+i: 0  k*p+j: 4  k,i,j 0 0 4
int 1 k*p+i: 1  k*p+j: 0  k,i,j 0 1 0
int 1 k*p+i: 1  k*p+j: 1  k,i,j 0 1 1
int 1 k*p+i: 1  k*p+j: 2  k,i,j 0 1 2
int 1 k*p+i: 1  k*p+j: 3  k,i,j 0 1 3
int 1 k*p+i: 1  k*p+j: 4  k,i,j 0 1 4
int 1 k*p+i: 2  k*p+j: 0  k,i,j 0 2 0
int 1 k*p+i: 2  k*p+j: 1  k,i,j 0 2 1
int 1 k*p+i: 2  k*p+j: 2  k,i,j 0 2 2
int 1 k*p+i: 2  k*p+j: 3  k,i,j 0 2 3
int 1 k*p+i: 2  k*p+j: 4  k,i,j 0 2 4
int 1 k*p+i: 3  k*p+j: 0  k,i,j 0 3 0
int 1 k*p+i: 3  k*p+j: 1  k,i,j 0 3 1
int 1 k*p+i: 3  k*p+j: 2  k,i,j 0 3 2
int 1 k*p+i: 3  k*p+j: 3  k,i,j 0 3 3
int 1 k*p+i: 3  k*p+j: 4  k,i,j 0 3 4
int 1 k*p+i: 4  k*p+j: 0  k,i,j 0 4 0
int 1 k*p+i: 4  k*p+j: 1  k,i,j 0 4 1
int 1 k*p+i: 4  k*p+j: 2  k,i,j 0 4 2
int 1 k*p+i: 4  k*p+j: 3  k,i,j 0 4 3
int 1 k*p+i: 4  k*p+j: 4  k,i,j 0 4 4
int 2 k*p+i: 4  k*p+j: 4  k,i,j 1 0 0
int 2 k*p+i:

In [82]:
B

array([[ 3.11373337, -1.37353571,  0.31412673, -0.23511992,  0.04760573,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-1.37353571,  3.84908852, -1.14565545,  0.92518683, -0.19423669,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.31412673, -1.14565545,  0.37619861, -0.31002267,  0.06567038,
         0.        ,  0.        ,  0.        ,  0